In [1]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_excel("bateria.xlsx")
data = data.iloc[:,1:2]
data.head()

,RAMJA Results
0,Enterococcus faecalis
1,NaN
2,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman..."
3,"K.pneumoniae, E. faecalis, P. mirabilis, A. b..."
4,NaN


In [10]:
names = data['RAMJA Results']

pandas.core.series.Series

In [ ]:
def cleanup(name):
    name = name.replace(";","")
    name = name.strip()
    name = name.capitalize()
    return name